About the Dataset:

1.id: unique id for a news article

2.title: the title of a news article

3.author: author of the news article

4.text: the text of the article; could 
be incomplete

5.label: a label that marks whether the news article is real or fake:



    1: Fake news
    0: real News

In [16]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [17]:
import nltk 
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [18]:
#printing the stopwords in English 
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

Data pre Processing

In [22]:
# loading the dataset to a pandas DataFrame
news_dataset = pd.read_csv('/content/train.csv',encoding='latin-1')


In [23]:
news_dataset.shape
news_dataset.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didnât Even See Comeyâs...,Darrell Lucus,House Dem Aide: We Didnât Even See Comeyâs...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [24]:
# counting the number of missing values in the dataset
news_dataset.isnull().sum()

# replacing the null values with empty string
news_dataset = news_dataset.fillna('')

In [25]:
# merging the author name and news title
news_dataset['content'] = news_dataset['author']+' '+news_dataset['title']

In [26]:
print(news_dataset['content'])

0       Darrell Lucus House Dem Aide: We Didnât Even...
1       Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo...
2       Consortiumnews.com Why the Truth Might Get You...
3       Jessica Purkiss 15 Civilians Killed In Single ...
4       Howard Portnoy Iranian woman jailed for fictio...
                              ...                        
9575    Jerome Hudson Rapper T.I.: Trump a âPoster C...
9576    Benjamin Hoffman N.F.L. Playoffs: Schedule, Ma...
9577    Michael J. de la Merced and Rachel Abrams Macy...
9578    Alex Ansary NATO, Russia To Hold Parallel Exer...
9579              David Swanson What Keeps the F-35 Alive
Name: content, Length: 9580, dtype: object


In [27]:
# separating the data & label
X = news_dataset.drop(columns='label', axis=1)
Y = news_dataset['label']

In [28]:
print(X)
print(Y)

         id                                              title  \
0         0  House Dem Aide: We Didnât Even See Comeyâs...   
1         1  FLYNN: Hillary Clinton, Big Woman on Campus - ...   
2         2                  Why the Truth Might Get You Fired   
3         3  15 Civilians Killed In Single US Airstrike Hav...   
4         4  Iranian woman jailed for fictional unpublished...   
...     ...                                                ...   
9575  20795  Rapper T.I.: Trump a âPoster Child For White...   
9576  20796  N.F.L. Playoffs: Schedule, Matchups and Odds -...   
9577  20797  Macyâs Is Said to Receive Takeover Approach ...   
9578  20798  NATO, Russia To Hold Parallel Exercises In Bal...   
9579  20799                          What Keeps the F-35 Alive   

                                         author  \
0                                 Darrell Lucus   
1                               Daniel J. Flynn   
2                            Consortiumnews.com   
3  

Stemming:

Stemming is the process of reducing a word to its Root word

example: actor, actress, acting --> act  (اختصار )

In [39]:
port_stem = PorterStemmer()
def stemming(content):
    stemmed_content = re.sub('[^a-zA-Z]',' ',content)
    stemmed_content = stemmed_content.lower()
    stemmed_content = stemmed_content.split()
    stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
    stemmed_content = ' '.join(stemmed_content)
    return stemmed_content

In [40]:
news_dataset['content'] = news_dataset['content'].apply(stemming)

In [42]:
print(news_dataset['content'])

0       darrel lucu hou dem aid even see comey letter ...
1       daniel j flynn flynn hillari clinton big woman...
2                  consortiumnew com truth might get fire
3       jessica purkiss civilian kill singl us airstri...
4       howard portnoy iranian woman jail fiction unpu...
                              ...                        
9575    jerom hudson rapper trump poster child white s...
9576    benjamin hoffman n f l playoff schedul matchup...
9577    michael j de la merc rachel abram maci said re...
9578    alex ansari nato russia hold parallel exerci b...
9579                            david swanson keep f aliv
Name: content, Length: 9580, dtype: object


In [45]:
#seprating data and label

X = news_dataset['content'].values
Y = news_dataset['label'].values
print(Y)
print(X)

[1 0 1 ... 0 1 1]
['darrel lucu hou dem aid even see comey letter jason chaffetz tweet'
 'daniel j flynn flynn hillari clinton big woman campu breitbart'
 'consortiumnew com truth might get fire' ...
 'michael j de la merc rachel abram maci said receiv takeov approach hudson bay new york time'
 'alex ansari nato russia hold parallel exerci balkan'
 'david swanson keep f aliv']


In [46]:
# converting the textual data to numerical data
vectorizer = TfidfVectorizer()
vectorizer.fit(X)

X = vectorizer.transform(X)
print(X)

  (0, 11176)	0.2871289004155165
  (0, 9598)	0.2554693780709465
  (0, 6360)	0.35884797828981885
  (0, 6149)	0.29770526027229904
  (0, 5485)	0.25483464733649286
  (0, 4998)	0.2153262253616258
  (0, 3545)	0.233456098879201
  (0, 2690)	0.27447016731233315
  (0, 2542)	0.35224361732679194
  (0, 2069)	0.24895646546909808
  (0, 1728)	0.3666592793736521
  (0, 178)	0.27065934205314507
  (1, 11955)	0.30767677479011424
  (1, 4858)	0.19537067792985424
  (1, 3943)	0.7097605872221622
  (1, 2523)	0.26475240493693525
  (1, 1964)	0.19573161052187615
  (1, 1548)	0.37165757093296714
  (1, 1305)	0.1610150731850931
  (1, 1025)	0.3015713948372116
  (2, 11118)	0.405003252589598
  (2, 6866)	0.4958242477308032
  (2, 4283)	0.3513560164252818
  (2, 3857)	0.3887563656431675
  (2, 2177)	0.46327141161097374
  :	:
  (9577, 9365)	0.2536914374668621
  (9577, 8805)	0.2742890217158662
  (9577, 8652)	0.2547046267661849
  (9577, 7349)	0.08239461969445622
  (9577, 6844)	0.1792456560754444
  (9577, 6791)	0.28607966694255854


In [47]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, stratify=Y, random_state=2)

In [48]:
model = LogisticRegression()

In [49]:
model.fit(X_train , Y_train)

LogisticRegression()

In [ ]:
#accuracy on training data
X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(X_train_prediction , Y_train)
print('Accuracy on training data :',training_data_accuracy)

In [50]:
#accuracy on training data
X_test_prediction = model.predict(X_test)
test_data_accuracy = accuracy_score(X_test_prediction , Y_test)
print('Accuracy on training data :',test_data_accuracy)

Accuracy on training data : 0.9624217118997912


In [52]:
X_new = X_test[20]



prediction = model.predict(X_new)
print(prediction)

if (prediction[0]==0):
  print('The news is Real')
else:
  print('The news is Fake')

[0]
The news is Real
